In [1]:
import pandas as pd
import os
import csv

In [2]:
# Creating a path
#movie_data = os.path.join("movies_metadata","ratings_small","wikipedia-movies.json")

In [4]:
# Read the data
#movies_lists_df = pd.read_csv(movie_data)
#movies_lists_df.head()

In [6]:
def extract_transform_load():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv("movies_metadata.csv")
    ratings = pd.read_csv("ratings_small.csv")

    # 3. Open the read the Wikipedia data JSON file.
    with open ("wikipedia-movies.json", mode='r') as file:
        wiki_movies_raw = json.load(file)
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings